<a href="https://colab.research.google.com/github/Aum-20/Aum_idenhq/blob/main/Aum_Chhajed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install playwright

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 MB 19.1 MB/s eta 0:00:00


In [2]:
!playwright install chromium

164.9 MiB [] 0% 0.0s164.9 MiB [] 0% 279.4s164.9 MiB [] 0% 262.2s164.9 MiB [] 0% 167.7s164.9 MiB [] 0% 156.3s164.9 MiB [] 0% 116.7s164.9 MiB [] 0% 99.9s164.9 MiB [] 0% 84.3s164.9 MiB [] 0% 86.8s164.9 MiB [] 0% 83.9s164.9 MiB [] 0% 51.9s164.9 MiB [] 0% 39.3s164.9 MiB [] 0% 39.9s164.9 MiB [] 1% 26.7s164.9 MiB [] 1% 23.5s164.9 MiB [] 1% 21.9s164.9 MiB [] 2% 18.9s164.9 MiB [] 2% 19.1s164.9 MiB [] 2% 18.8s164.9 MiB [] 2% 17.6s164.9 MiB [] 2% 16.7s164.9 MiB [] 3% 15.7s164.9 MiB [] 3% 13.9s164.9 MiB [] 3% 13.8s164.9 MiB [] 3% 13.7s164.9 MiB [] 3% 13.6s164.9 MiB [] 4% 12.3s164.9 MiB [] 5% 11.2s164.9 MiB [] 5% 10.5s164.9 MiB [] 5% 9.9s164.9 MiB [] 6% 9.3s164.9 MiB [] 6% 9.6s164.9 MiB [] 6% 9.5s164.9 MiB [] 7% 9.3s164.9 MiB [] 7% 9.4s164.9 MiB [] 7% 9.3s164.9 MiB [] 8% 9.0s164.9 MiB [] 8% 8.6s164.9 MiB [] 8% 8.4s164.9 MiB [] 9% 8.3s164.9 MiB [] 9% 8.4s164.9 MiB [] 10% 8.1s164.9 MiB [] 10% 8.0s164.9 MiB [] 10% 7.9s164.9 MiB [] 11% 7.7s164.9 MiB [] 11% 7.4s164.9 MiB [] 12% 7.2s164.9 MiB [] 13% 7.1s

In [5]:
import asyncio
import json
import os
from playwright.async_api import async_playwright, TimeoutError
import nest_asyncio

# Apply nest_asyncio to allow nested event loops
nest_asyncio.apply()

async def main():
    session_file = "session.json"
    output_file = "product_data.json"

    email = "chhajed.aum@gmail.com"
    password = "JGSMDiux"

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context()

        session_exists = False
        if os.path.exists(session_file):
            try:
                with open(session_file, "r") as f:
                    storage_state = json.load(f)
                    await context.set_storage_state(state=storage_state)
                    session_exists = True
                    print("Using existing session.")
            except Exception as e:
                print(f"Error loading session: {e}")

        page = await context.new_page()
        print("Navigating to challenge page...")
        await page.goto("https://hiring.idenhq.com")

        # Increase timeout for wait_for_selector to 120 seconds
        if not session_exists or await page.is_visible("text=Sign in"):
            print("No valid session found. Logging in...")
            await page.wait_for_selector("input[type='email']", timeout=120000)  # Increased timeout
            await page.fill("input[type='email']", email)
            await page.fill("input[type='password']", password)
            await page.click("button:has-text('Sign in')")
            await page.wait_for_load_state("networkidle")

            storage_state = await context.storage_state()
            with open(session_file, "w") as f:
                f.write(json.dumps(storage_state))
            print("Session saved.")

        # Save screenshot after login
        await page.screenshot(path="after_login.png")
        print("Screenshot taken after login.")

        try:

            print("Clicking 'Launch'...")
            await page.click("button:has-text('Launch Challenge')", timeout=120000)

            print("Clicking 'Open Dashboard Menu'...")
            await page.click("button:has-text('Open Dashboard Menu')", timeout=120000)

            print("Clicking 'Data Tools'...")
            await page.click("text=Data Tools", timeout=120000)

            print("Clicking 'Data Tools'...")
            await page.click("text=Data Tools", timeout=120000)

            print("Clicking 'Inventory Options'...")
            await page.wait_for_selector("button:has-text('Inventory Options')", timeout=120000)
            await page.click("button:has-text('Inventory Options')")

            print("Clicking 'Open Products Drawer'...")
            await page.click("text=Open Products Drawer", timeout=120000)

    # Wait for the table to load (increased timeout)
            await page.wait_for_selector(".product-table", timeout=300000)
        except TimeoutError as te:
            await page.screenshot(path="timeout_error.png")
            print(f"Timeout occurred: {te}")
            print("Saved screenshot to 'timeout_error.png' for debugging.")
            await browser.close()
            return

        all_products = []

        async def extract_products():
            return await page.evaluate("""() => {
                const products = [];
                const rows = document.querySelectorAll('.product-row');
                rows.forEach(row => {
                    const product = {};
                    product.id = row.querySelector('.product-id') ? row.querySelector('.product-id').textContent.trim() : '';
                    product.name = row.querySelector('.product-name') ? row.querySelector('.product-name').textContent.trim() : '';
                    product.category = row.querySelector('.product-category') ? row.querySelector('.product-category').textContent.trim() : '';
                    product.composition = row.querySelector('.product-composition') ? row.querySelector('.product-composition').textContent.trim() : '';
                    product.weight = row.querySelector('.product-weight') ? parseFloat(row.querySelector('.product-weight').textContent.trim()) : 0;
                    product.cost = row.querySelector('.product-cost') ? parseFloat(row.querySelector('.product-cost').textContent.trim().replace('$', '')) : 0;
                    product.manufacturer = row.querySelector('.product-manufacturer') ? row.querySelector('.product-manufacturer').textContent.trim() : '';
                    product.shade = row.querySelector('.product-shade') ? row.querySelector('.product-shade').textContent.trim() : '';
                    product.guarantee = row.querySelector('.product-guarantee') ? row.querySelector('.product-guarantee').textContent.trim() : '';
                    product.updated = row.querySelector('.product-updated') ? row.querySelector('.product-updated').textContent.trim() : '';
                    products.push(product);
                });
                return products;
            }""")

        print("Checking for pagination...")
        has_pagination = await page.is_visible(".pagination")

        if has_pagination:
            while True:
                products = await extract_products()
                all_products.extend(products)
                if await page.is_visible("button.next-page:not([disabled])"):
                    await page.click("button.next-page")
                    await page.wait_for_load_state("networkidle")
                else:
                    break
        else:
            # Extract products if no pagination
            products = await extract_products()
            all_products.extend(products)

        # Save extracted data to output file
        with open(output_file, "w") as f:
            json.dump(all_products, f, indent=2)

        print(f"Successfully extracted {len(all_products)} products to {output_file}")
        await browser.close()

# For Jupyter notebooks
await main()

Error loading session: 'BrowserContext' object has no attribute 'set_storage_state'
Navigating to challenge page...
No valid session found. Logging in...
Session saved.
Screenshot taken after login.
Clicking 'Launch'...
Clicking 'Open Dashboard Menu'...
Clicking 'Data Tools'...
Clicking 'Data Tools'...
Clicking 'Inventory Options'...
Clicking 'Open Products Drawer'...
Timeout occurred: Page.wait_for_selector: Timeout 300000ms exceeded.
Call log:
  - waiting for locator(".product-table") to be visible

Saved screenshot to 'timeout_error.png' for debugging.
